# HyperDrive demo notebook

This notebook will demostrate the use of the AzureML tool HyperDrive which allows for distributed hyperparameter tuning.

 Hyperparameters are used to control the training of a machine learning model and hyperparameter tuning is the problem of choosing a set of optimal hyperparameters for a learning algorithm for the specific application. This requires compiling and training a model over and over with different combinations of hyperparameters in a defined hyperparameter space, so the process can be very time intensive. Parallelising this processing can significantly increase the time efficiency of this process. 

The environment that this notebook works best with is: <b>Python 3.8 - AzureML</b>

In [41]:
%matplotlib inline

In [2]:
import tempfile

In [40]:
import mlflow

In [3]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt

In [5]:
import prd_pipeline

## Set up azure experiment


In [6]:
import azureml.core
from azureml.core import Workspace, Datastore, Dataset, Environment
from azureml.core import Experiment, ComputeTarget, ScriptRunConfig

In [7]:
prd_ws = Workspace.from_config()

In [8]:
azure_experiment_name='prd_fraction_models'
azure_env_name = 'prd_ml_cluster'

In [9]:
prd_model_name = 'azml_cluster_demo_202208'

In [10]:
use_full_dataset = True
if use_full_dataset:
    azure_dataset_name ='prd_merged_all_events_files'
    prd_blob_rel_path = 'prd/*/prd_merged*csv'    
else:
    #  use subset for development.
    azure_dataset_name ='prd_merged_202110_nswws_amber_oct_files'
    prd_blob_rel_path = 'prd/202110_nswws_amber_oct/prd_merged*csv'    


In [11]:
use_gpu = True
if use_gpu:
    do_download_data = True # if false, use the azml dataset, otherwise download from datastore or blob store
    use_blob_store = True # only applies if do download is true. If this is true, download from the blob store rather than the datastore.
else:
    do_download_data = False # if false, use the azml dataset, otherwise download from datastore or blob store
    use_blob_store = False # only applies if do download is true. If this is true, download from the blob store rather than the datastore.

In [12]:
# this env has fsspec and related to facilitate loading data on GPU instance
azure_env_name = 'prd_ml_gpu_cluster'

In [13]:
if use_gpu:
    if use_full_dataset:
        cluster_name = 'prd-ml-fractions-cluster-gpu'
    else:
        cluster_name = 'mlops-gpu-test'
else:
    if use_full_dataset:
        cluster_name = 'prd-ml-fractions-cluster'
    else:
        cluster_name = 'mlops-test'
cluster_name

'prd-ml-fractions-cluster-gpu'

In [ ]:
target_parameter = [
    'radar_fraction_in_band_instant_0.0',
    'radar_fraction_in_band_instant_0.25',
    'radar_fraction_in_band_instant_2.5',
    'radar_fraction_in_band_instant_7.0',
    'radar_fraction_in_band_instant_10.0'
]
profile_features = ['air_temperature', 'relative_humidity']
single_lvl_features = []

In [ ]:
feature_dict = {
    'profile': profile_features,
    'single_level': single_lvl_features,
    'target': target_parameter,
} 

In [ ]:
prd_exp = Experiment(workspace=prd_ws, name=azure_experiment_name)
prd_exp

Name,Workspace,Report Page,Docs Page
prd_fraction_models,precip_rediagnosis,Link to Azure Machine Learning studio,Link to Documentation


Get the AzML environment (basically a conda environment) from the workspace.

In [ ]:
prd_env = Environment.get(workspace=prd_ws, name=azure_env_name)
prd_env

{
    "assetId": "azureml://locations/uksouth/workspaces/57546dc9-9763-4025-831d-c19991c81540/environments/prd_ml_gpu_cluster/versions/4",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.2-cudnn8-ubuntu18.04",
        "baseImageRegistry": null,
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {},
    "inferencingStackVersion": null,
    "name": "prd_ml_gpu_cluster",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "conda-forge"
            ],
            "depen

### Execute our training run on a cluster with hyperdrive for parallelised hyperparameter tuning

In [ ]:
import datetime
log_dir = 'logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

In [ ]:
prd_demo_compute_target = ComputeTarget(workspace=prd_ws, name=cluster_name)
prd_demo_compute_target

AmlCompute(workspace=Workspace.create(name='precip_rediagnosis', subscription_id='07efdc52-cd27-48ed-9443-3aad2b6b777b', resource_group='precip_rediagnosis'), name=prd-ml-fractions-cluster-gpu, id=/subscriptions/07efdc52-cd27-48ed-9443-3aad2b6b777b/resourceGroups/precip_rediagnosis/providers/Microsoft.MachineLearningServices/workspaces/precip_rediagnosis/computes/prd-ml-fractions-cluster-gpu, type=AmlCompute, provisioning_state=Succeeded, location=uksouth, tags={})

Hyperparameters that we want to vary using hyperdrive need to be input arguments for the prd_cluster_train_demo.py script which is called through ScriptRunConfig. Hyperparameters set in prd_demo_args will be overwritten by Hyperdrive. 

In [ ]:
nepochs = 1

In [22]:
prd_demo_args = ['--dataset-name', azure_dataset_name,
                 '--model-name', prd_model_name,
                ]

In [ ]:
prd_demo_args += ['--target-parameter']
prd_demo_args += target_parameter
prd_demo_args += ['--profile-features']
prd_demo_args += profile_features
prd_demo_args += ['--single-level_features']
prd_demo_args += single_lvl_features
prd_demo_args += ['--epochs', nepochs]
prd_demo_args += ['--batch-size', 128]
prd_demo_args += ['--learning-rate', 0.01]
prd_demo_args += ['--test-fraction', 0.2]
prd_demo_args += ['--log-dir', './logs']
prd_demo_args += ['--autolog']

In [24]:
if do_download_data:
    if use_blob_store:
        prd_demo_args += ['--data-path', prd_blob_rel_path]
        prd_demo_args += ['--blob']    
    else:
        prd_demo_args += ['--data-path', azureml.core.Dataset.get_by_name(prd_ws, azure_dataset_name).as_download()]

In [25]:
prd_demo_args

['--dataset-name',
 'prd_merged_all_events_files',
 '--model-name',
 'azml_fractions_cluster_hpt',
 '--target-parameter',
 'radar_fraction_in_band_instant_0.0',
 'radar_fraction_in_band_instant_0.25',
 'radar_fraction_in_band_instant_2.5',
 'radar_fraction_in_band_instant_7.0',
 'radar_fraction_in_band_instant_10.0',
 '--profile-features',
 'air_temperature',
 'relative_humidity',
 '--single-level_features',
 '--epochs',
 1,
 '--batch-size',
 128,
 '--learning-rate',
 0.01,
 '--test-fraction',
 0.2,
 '--log-dir',
 './logs',
 '--autolog',
 '--data-path',
 'prd/*/prd_merged*csv',
 '--blob']

In [ ]:
prd_run_src = ScriptRunConfig(
    source_directory=os.getcwd(),
    script='prd_cluster_train_demo.py',
    arguments=prd_demo_args,
    compute_target=prd_demo_compute_target,
    environment=prd_env
)

In [24]:
prd_run = prd_exp.submit(prd_run_src)
prd_run

Experiment,Id,Type,Status,Details Page,Docs Page
prd_fraction_models,prd_fraction_models_1665149501_02b742c4,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


In [25]:
prd_run.wait_for_completion()
assert(prd_run.get_status() == "Completed")

### HyperDrive configuration

The next step is to configure our HyperDrive run. The run config defined above is passed to our HyperDriveConfig as the run_config. We must provide a <code>hyperparameter_sampling</code> explained in more detail below. We are required to also provide the <code>primary_metric_name</code> (either be the models loss function or a metric define when compiling the model) and the <code>primary_metric_goal</code> to either minimize or maximize this metric. We must also define <code>max_total_runs</code> (the upper bound of the number of runs, may be smaller depending on the defined hyperparameter space and sampling strategy) and <code>max_concurrent_runs</code>, if this is set to None all run are launched in parallel.
There is also the option to define an early stopping policy with the <code>policy</code> argument.

In [26]:
from azureml.train.hyperdrive import RandomParameterSampling, GridParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform

#### Hyperparameter sampling

There are three different classes for sampling the hyperparameter space: 
- <code>GridParameterSampling</code>: define a search space as a grid of hyperparameter based on the given hyperparameter space, then evaluates every position in the grid in order (note if max_total_runs < total potential combinations, then this will only run a subsample of the grid).
- <code>RandomParameterSampling</code>: randomly samples hyperparameter combinations from the hyperparameter space 
- <code>BayesianParameterSampling</code>: defines Bayesian sampling over a hyperparameter space, tries to intelligently pick the next sample of hyperparameters based on how the previous samples performed.

We then define the hyperparameter space from which to select the combination of hyperparameters to assess. The hyperparameters that we want to tune should be input arguments to the script which is being run by ScriptRunConfig and any hyperparameter arguments that are input will be overwritten by hyperdrive.

In AzureML, there are different ways to define the set to sample each hyperparameters from (<code>choice</code>, <code>lognormal</code>, <code>loguniform</code>, <code>normal</code>, <code>qlognormal</code>, <code>qloguniform</code>, <code>qnormal</code>, <code>quniform</code>, <code>randint</code> and <code>uniform</code>).

In [27]:
ps = RandomParameterSampling(
    {
        '--batch-size': choice(32),
        '--learning-rate': choice(0.1, 0.01),
        '--epochs': choice(10, 20, 50, 100)
    }
)

#### Early stopping policy

We can define an early stopping policy in which means that any poorly performing experiment runs are canceled and new ones started. Here we use BanditPolicy with a slack criteria of 0.1 (the ratio of slack allowed with respect to the best performing training run) and evaluation internal of 2 (the frequency for applying the policy, here every two training steps).

In [28]:
early_stop_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

In [29]:
htc = HyperDriveConfig(run_config=prd_run_src, 
                       hyperparameter_sampling=ps, 
                       policy=early_stop_policy, 
                       primary_metric_name='val_loss', 
                       primary_metric_goal=PrimaryMetricGoal.MINIMIZE, 
                       max_total_runs=18,
                       max_concurrent_runs=12)

### HyperDrive running

In [30]:
prd_run_src

In [31]:
prd_run = prd_exp.submit(htc)
prd_run

Experiment,Id,Type,Status,Details Page,Docs Page
prd_fraction_models,HD_2d856d18-157c-4abc-9e10-4f8fa8d038a7,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


Experiment,Id,Type,Status,Details Page,Docs Page
prd_fraction_models,HD_2d856d18-157c-4abc-9e10-4f8fa8d038a7,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [32]:
import azureml.tensorboard

In [33]:
tb = azureml.tensorboard.Tensorboard([prd_run])

In [34]:
# If successful, start() returns a string with the URI of the instance.
tb.start()

https://prd-ml-large-6006.uksouth.instances.azureml.ms


'https://prd-ml-large-6006.uksouth.instances.azureml.ms'

In [35]:
prd_run.wait_for_completion()
assert(prd_run.get_status() == "Completed")

### HyperDrive results

We can then select the model which performs best against the selected primary metric, as defined within the HyperDriveConfig.

Get the best trained model from the hyper drive run and load in the trained weights ready for inference.

In [36]:
prd_best_run = prd_run.get_best_run_by_primary_metric()
prd_best_run

In [37]:
prd_tracking_uri = prd_ws.get_mlflow_tracking_uri()
prd_tracking_uri

'azureml://uksouth.api.azureml.ms/mlflow/v1.0/subscriptions/07efdc52-cd27-48ed-9443-3aad2b6b777b/resourceGroups/precip_rediagnosis/providers/Microsoft.MachineLearningServices/workspaces/precip_rediagnosis?'

In [42]:
mlflow.set_tracking_uri(prd_tracking_uri)

In [43]:
%%time
trained_model = mlflow.keras.load_model(f'runs:/{prd_best_run.id}/model')

2022-10-05 16:23:51.622774: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


CPU times: user 535 ms, sys: 46.9 ms, total: 582 ms
Wall time: 2.49 s


In [44]:
trained_model

### Load the data for inference

Load the data from the same loading functions as in the script for consistencyt

In [45]:
import json

In [46]:
with open('credentials_file.json') as credentials_file:
    az_blob_cred = json.load(credentials_file)
    
az_blob_cred.keys()

dict_keys(['storage_acc_name', 'storage_acc_key', 'container'])

In [51]:
%%time
input_data = prd_pipeline.load_data_azure_blob(
    az_blob_cred=az_blob_cred,
    blob_path=prd_blob_rel_path,
)


loading data direct from blobstore
CPU times: user 27 s, sys: 2.81 s, total: 29.9 s
Wall time: 57.9 s


In [52]:
do_save_split = False

In [53]:
%%time
if do_save_split:
    data_splits, data_dims = prd_pipeline.preprocess_data(
        input_data,
        test_fraction=0.2,
        feature_dict={'profile': profile_features, 'single_level': single_lvl_features,'target': target_parameter,},
        test_savefn='tmp.csv',
    )
else:
    data_splits, data_dims = prd_pipeline.preprocess_data(
        input_data,
        test_fraction=0.2,
        feature_dict={'profile': profile_features, 'single_level': single_lvl_features,'target': target_parameter,},
    )
    

target has dims: 5
dropping smallest bin: radar_fraction_in_band_instant_0.0
getting profile columns
{'nprof_features': 2, 'nheights': 33, 'nsinglvl_features': 0, 'nbands': 5}
CPU times: user 2.39 s, sys: 1.99 s, total: 4.38 s
Wall time: 4.48 s


In [54]:
data_splits['X_train']

array([[[ 0.57866682,  1.40322659],
        [ 0.57866682,  1.39294129],
        [ 0.57838692,  1.36989504],
        ...,
        [-1.52183695,  0.74598158],
        [-1.82387533,  0.65797094],
        [-1.6108297 ,  0.54260528]],

       [[ 0.70889926,  1.29350284],
        [ 0.70889926,  1.2834819 ],
        [ 0.7086016 ,  1.23945299],
        ...,
        [-2.11978288,  0.72052686],
        [-1.86093381,  0.56731207],
        [-1.62902172,  0.44409824]],

       [[ 0.77401572,  1.29350284],
        [ 0.77401572,  1.2834819 ],
        [ 0.77370918,  1.23945299],
        ...,
        [ 0.62774081,  0.70355705],
        [-0.80847349,  0.60852065],
        [-1.08326069,  0.5097696 ]],

       ...,

       [[-1.21668401, -0.22068503],
        [-1.21668401, -0.24894945],
        [-1.21671908, -0.32585172],
        ...,
        [-0.08373918, -1.19706206],
        [ 0.04757706, -1.27883211],
        [ 0.08102937, -1.42753553]],

       [[-0.05389237, -0.68152481],
        [-0.05389237, -0.70

We can also return information from each of the different hyperdrive child run

In [38]:
mlflow.set_tracking_uri(prd_tracking_uri)

In [39]:
%%time
trained_model = mlflow.keras.load_model(f'runs:/{prd_best_run.id}/model')

2022-10-07 14:10:33.809063: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-10-07 14:10:34.027938: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-07 14:10:34.028001: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (prd-ml-fractions): /proc/driver/nvidia/version does not exist
2022-10-07 14:10:34.031169: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-10-07 14:10:34.105247: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2095195000 Hz
2022-10-07 14:10:34.108631: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f1524000b60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [40]:
trained_model

NameError: name 'trained_model' is not defined

### Load the data for inference

Load the data from the same loading functions as in the script for consistency

In [ ]:
import json

In [ ]:
with open('credentials_file.json') as credentials_file:
    az_blob_cred = json.load(credentials_file)
    
az_blob_cred.keys()

In [ ]:
%%time
input_data = prd_pipeline.load_data_azure_blob(
    az_blob_cred=az_blob_cred,
    blob_path=prd_blob_rel_path,
)

In [ ]:
do_save_split = False

In [ ]:
%%time
if do_save_split:
    data_splits, data_dims = prd_pipeline.preprocess_data(
        input_data,
        test_fraction=0.2,
        feature_dict={'profile': profile_features, 'single_level': single_lvl_features,'target': target_parameter,},
        test_savefn='tmp.csv',
    )
else:
    data_splits, data_dims = prd_pipeline.preprocess_data(
        input_data,
        test_fraction=0.2,
        feature_dict={'profile': profile_features, 'single_level': single_lvl_features,'target': target_parameter,},
    )

In [55]:
y_pred = trained_model.predict(data_splits['X_val'])

In [56]:
y_pred

array([[6.6825223e-01, 2.4224499e-01, 7.8887478e-02, 8.6118234e-03,
        2.0034988e-03],
       [2.5721881e-01, 6.3474637e-01, 1.0713762e-01, 8.6187338e-04,
        3.5354296e-05],
       [9.0825933e-01, 9.0753183e-02, 9.8433951e-04, 3.1369789e-06,
        3.1342605e-08],
       ...,
       [6.1969036e-01, 2.1871240e-01, 1.5794353e-01, 3.5361594e-03,
        1.1751348e-04],
       [4.0459248e-01, 1.6454746e-01, 3.8138270e-01, 4.8330806e-02,
        1.1465524e-03],
       [6.0364330e-01, 1.2005442e-01, 2.2921899e-01, 4.6148367e-02,
        9.3489635e-04]], dtype=float32)

In [ ]:
prd_run.get_children_sorted_by_primary_metric()

[{'run_id': 'HD_19ec7c9b-0557-48a0-9429-65f9fdcdeaae_10',
  'hyperparameters': '{"--batch-size": 64, "--epochs": 20, "--learning-rate": 0.001}',
  'best_primary_metric': 0.4036311209201813,
  'status': 'Completed'},
 {'run_id': 'HD_19ec7c9b-0557-48a0-9429-65f9fdcdeaae_2',
  'hyperparameters': '{"--batch-size": 128, "--epochs": 100, "--learning-rate": 0.001}',
  'best_primary_metric': 0.4036446809768677,
  'status': 'Completed'},
 {'run_id': 'HD_19ec7c9b-0557-48a0-9429-65f9fdcdeaae_17',
  'hyperparameters': '{"--batch-size": 32, "--epochs": 50, "--learning-rate": 0.001}',
  'best_primary_metric': 0.4075716733932495,
  'status': 'Completed'},
 {'run_id': 'HD_19ec7c9b-0557-48a0-9429-65f9fdcdeaae_6',
  'hyperparameters': '{"--batch-size": 128, "--epochs": 50, "--learning-rate": 0.001}',
  'best_primary_metric': 0.41052529215812683,
  'status': 'Completed'},
 {'run_id': 'HD_19ec7c9b-0557-48a0-9429-65f9fdcdeaae_5',
  'hyperparameters': '{"--batch-size": 64, "--epochs": 100, "--learning-rate"

In [ ]:
prd_run.get_metrics()

{'HD_19ec7c9b-0557-48a0-9429-65f9fdcdeaae_17': {'loss': [0.49701470136642456,
   0.45489686727523804,
   0.4338328540325165,
   0.41986083984375,
   0.40857386589050293,
   0.4003901183605194,
   0.39313700795173645,
   0.3870382010936737,
   0.3818308711051941,
   0.37716931104660034,
   0.3726852536201477,
   0.3690747022628784,
   0.3654104769229889,
   0.36236807703971863,
   0.35901519656181335,
   0.35674363374710083,
   0.3545750081539154,
   0.3517545461654663,
   0.3490726053714752,
   0.3474254608154297,
   0.3453293740749359,
   0.34355825185775757,
   0.34212154150009155,
   0.3404567241668701,
   0.3391324281692505,
   0.3372246026992798,
   0.33583390712738037,
   0.3345494568347931,
   0.33350542187690735,
   0.33239805698394775,
   0.33127233386039734,
   0.3304336965084076,
   0.3290450870990753,
   0.328093558549881,
   0.32708677649497986,
   0.32592588663101196,
   0.3248363435268402,
   0.3244028389453888,
   0.3227832615375519,
   0.3227120041847229,
   0.32177519

In [59]:
prd_run.get_hyperparameters()

{'HD_19ec7c9b-0557-48a0-9429-65f9fdcdeaae_0': '{"--batch-size": 128, "--epochs": 10, "--learning-rate": 0.1}',
 'HD_19ec7c9b-0557-48a0-9429-65f9fdcdeaae_1': '{"--batch-size": 32, "--epochs": 20, "--learning-rate": 0.1}',
 'HD_19ec7c9b-0557-48a0-9429-65f9fdcdeaae_2': '{"--batch-size": 128, "--epochs": 100, "--learning-rate": 0.001}',
 'HD_19ec7c9b-0557-48a0-9429-65f9fdcdeaae_3': '{"--batch-size": 64, "--epochs": 100, "--learning-rate": 0.01}',
 'HD_19ec7c9b-0557-48a0-9429-65f9fdcdeaae_4': '{"--batch-size": 128, "--epochs": 50, "--learning-rate": 0.0001}',
 'HD_19ec7c9b-0557-48a0-9429-65f9fdcdeaae_5': '{"--batch-size": 64, "--epochs": 100, "--learning-rate": 0.0001}',
 'HD_19ec7c9b-0557-48a0-9429-65f9fdcdeaae_6': '{"--batch-size": 128, "--epochs": 50, "--learning-rate": 0.001}',
 'HD_19ec7c9b-0557-48a0-9429-65f9fdcdeaae_7': '{"--batch-size": 32, "--epochs": 100, "--learning-rate": 0.0001}',
 'HD_19ec7c9b-0557-48a0-9429-65f9fdcdeaae_8': '{"--batch-size": 32, "--epochs": 20, "--learning-ra